In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langgraph.graph import StateGraph,START,END
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
import base64
import uuid
import os
import io
import re
from IPython.display import display, HTML
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from dotenv import load_dotenv
from groq import Groq
load_dotenv()
client = Groq()

In [4]:
from unstructured.partition.pdf import partition_pdf

In [5]:
import os

def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    full_path = os.path.join(path,fname)
    return partition_pdf(
        filename=full_path,
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        image_output_dir_path=path,
    )

In [6]:
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables

In [7]:
fpath = "C:/Users/gokul/PROJECT_CAPSTONE/Rag_model"
fname = "rag_img_test.pdf"
# Get elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [11]:
texts_4k_token

["Inside 8.\n\nThey'll Do anything\n\n=\n\nch Sib During COVID-1'\n\nL ~\n\nx\n\n— = : ~s es\n\n,\n\nak\n\nlot an Ostrich’ A War in Pictures American Gothic\n\na = ¥ et —\n\nFEATURES\n\n02\n\n10\n\nIf\n\n9\n\nGreat Photographs\n\nImages from the Library’s collections hold the power to educate and inspire.\n\n‘Not an Ostrich’\n\nExhibit of images explores the whimsical and serious sides of American life.\n\nGetting the Shot\n\nPhotographers will do just about anything to capture the perfect image.\n\n‘American Gothic’\n\nParks chronicled the life of a Black worker — and created a masterpiece.\n\nLIBRARY OF CONGRESS MAGAZINE\n\nLIBRARY OF CONGRESS MAGAZINE\n\nNOVEMBER / DECEMBER 2020 VOL. 9 NO. 5\n\nMission of the Library of Congress\n\nThe Library’s mission is to engage, inspire and inform Congress and the American people with a universal and enduring source of knowledge and creativity.\n\nLibrary of Congress Magazine is.\n\nissued bimonthly by the Office of Communications of the Librar

In [12]:
tables

[]

In [25]:
fpath = "c:/Users/gokul/Downloads"
fname = "24MAI1024-Technical Report Writing(MENG501P)-1.pdf"
# Get elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Get text, tables
texts_t, tables_t= categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts_t = " ".join(texts_t)
texts_4k_token_t = text_splitter.split_text(joined_texts_t)

In [26]:
texts_4k_token_t

['Course Name: Technical Report Writing Course Code: MENGS01P Slot: L3+L4+L19+L20\n\nTitle: Discrimination\n\nDone By:\n\nName: Gokula Prasath S\n\nReg No: 24MAI1024\n\nDate: 21-01-2025 for\n\n7\n\ncomprehensi ve strategies. | Williams, David | Exploresthe | Situatesthe | How does Systematic Individuals Thematic and Callsformulti- | Addressing R,, Jourdyn A. impact of discussion racial review of 29 reporting integrative discrimination | level systemic Lawrence, racial within discrimination studies racial or review interventions, racism is Brigette A. discriminatio | Systemic impact mental} published ethnic categorizing independent cultural ctucial to explore Davis, and Racism, health, between 2013- | discriminatio | studies into risk factor for competency mitigate the intergeneration Cecilia Vu. focusingon | physical mental health, poor mental training, and long-term aleffects and pathways health, and focus on physical health, | and physical anti-racism health impacts | more innovative

In [22]:
tables_t

['S.No | Citation Purpose of Reasearch Supporting | Theory Research Question Research Method Population Analytical Technique Major findings Implication Conclusion ‘Summary - Research Gap 1 | Harriss-white, B., &Prakash,A. Social Discrimination in India: ACase for Economic Citizenship. Oxford University | The study | examineshow | social | identities like} | caste, religion, and gender drive unequal economic outcomes and proposes "economic citizenship" asa strategy to mitigate discriminatio n Social discriminatio | nis framed asan intergroup phenomenon | sustained by | institutional | and systemic | inequalities. How dosociat | identity-based | exclusions impact economic opportunities, | and what institutional measurescan | address these | disparities? A multidisciplina | ry approach uses secondary data | and policy analysis to explore pattems of discrimination and their implications. The focus is onhistorically | marginalized | groups in India, particularly Dalits, Adivasis, and Muslim

In [33]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(image_base64):

    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {
                "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""
            },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}", 
                        }
                    }
                ]
            }
        ],
    )

    return completion.choices[0].message.content

In [34]:
def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []


    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg") or img_file.endswith(".png"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image))

    return img_base64_list, image_summaries



In [35]:
def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


In [36]:
img_base64_list, image_summaries = generate_img_summaries("C:/Users/gokul/PROJECT_CAPSTONE/Rag_model")

In [37]:
image_summaries

["A pink-haired anime girl sitting on a briefcase next to a white van. She's wearing a black jacket, black boots and black thigh-high socks. Her hair is styled into two pigtails. She's on a city street. There are manhole covers and chain link fence on the sidewalk behind her. Her arms are folded. The image contains a yellow toy on her belt. The girl's style is goth/kawaii. A vending machine is on the right. The image seems to be a digitally rendered 3D image."]

In [38]:
vectorstore = Chroma(
    collection_name="RAG", embedding_function=HuggingFaceEmbeddings()
)

c:\Users\gokul\anaconda3\envs\langgraphenv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gokul\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [40]:
# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries



In [41]:

# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [42]:
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

In [43]:
def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))

In [44]:
def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None

In [45]:
def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False

In [46]:
def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [47]:
def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}

In [48]:
def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)
    text_message = {
        "type": "text",
        "text": (
            "You are financial analyst tasking with providing investment advice.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide investment advice related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]

In [49]:
def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


In [50]:
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [51]:
query ="Give me discrimination in math"
docs = retriever_multi_vector_img.invoke(query, limit=6)

In [52]:
docs

['NOVEMBER/DECEMBER 2020 LOC.GOV/LCM 3\n\nTRENDING\n\n= Camilo J. Vergara. “Buying paper towels, 82nd at Roosevelt Ave., Queens.” April 25, 2020. Digital photograph. © Camilo Vergara, used by permission. Prints and Photographs Division.\n\nCamilo J. Vergara. “George Floyd memorial, 884 Broadway, Brooklyn.” July 3, 2020. Digital photograph. Prints and Photographs Division.\n\nCOLLECTING CURRENT EVENTS\n\nThe Library documents contemporary issues such as COVID-19 and social justice.\n\nWhen the COVID-19 pandemic put the city that never sleeps to bed, photographer Camilo José Vergara’s dedication to documenting poor neighborhoods in the New York City region increased significantly.\n\nlocations with great population density and rich diversity, are visual proof of how some of the most vulnerable Americans responded to new orders to stay at home, social distance and wear masks. The inventiveness of small business owners are portrayed vividly and humanely, as are the pandemic’s effects on sh

In [55]:
# Check retrieval
query ="Covid 19"
docs = retriever_multi_vector_img.invoke(query, limit=6)


In [56]:
plt_img_base64(docs[0])

In [57]:
plt_img_base64(docs[1])

In [60]:
print(docs[0])

Inside 8.

They'll Do anything

=

ch Sib During COVID-1'

L ~

x

— = : ~s es

,

ak

lot an Ostrich’ A War in Pictures American Gothic

a = ¥ et —

FEATURES

02

10

If

9

Great Photographs

Images from the Library’s collections hold the power to educate and inspire.

‘Not an Ostrich’

Exhibit of images explores the whimsical and serious sides of American life.

Getting the Shot

Photographers will do just about anything to capture the perfect image.

‘American Gothic’

Parks chronicled the life of a Black worker — and created a masterpiece.

LIBRARY OF CONGRESS MAGAZINE

LIBRARY OF CONGRESS MAGAZINE

NOVEMBER / DECEMBER 2020 VOL. 9 NO. 5

Mission of the Library of Congress

The Library’s mission is to engage, inspire and inform Congress and the American people with a universal and enduring source of knowledge and creativity.

Library of Congress Magazine is.

issued bimonthly by the Office of Communications of the Library of Congress and distributed free of charge to publicly suppo